# DIA: How To Generate a Template Image
<br>Owner(s): **Phil Marshall** ([@drphilmarshall](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@drphilmarshall))
<br>Last Verified to Run: **2018-08-24**
<br>Verified Stack Release: **16.0**
<br>Discussion: **[issue #48](https://github.com/LSSTScienceCollaborations/DMStackClub/issues/48)**

In this tutorial we will understand the possible algorithm choices for DIA template image generation, see how these are currently implemented in the science pipelines, and demonstrate them on the Twinkles images.

### Learning Objectives:

After working through this tutorial you should be able to: 
1. Use a SkyMap to identify all the visit images in a given filter you want to coadd ;
2. Combine these images into a decorrelated template image.

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

During development, it's helpful to have the `stackclub` library available - and the `where_is` documentation finder in particular.

In [ ]:
! cd .. && python setup.py -q develop --user && cd -

In [ ]:
from stackclub import where_is

## Background

The DRP pipelines are summarized in Figure 9 of the _Data Management Science Pipelines
Design Document_, [LDM-151](http://ls.st/ldm-151), reproduced here:

![](https://github.com/lsst/LDM-151/raw/master/figures/drp_coaddition_and_diffim.png)

Difference imaging is shown in block 4, and includes creation of TemplateCoadd images, image differencing, and DIASource creation. The relevant pipeline design text on template generation, for the data release processing, is in Section 5.2:

> "4. We run the WarpTemplates, CoaddTemplates, and DiffIm pipelines to generate the DIASource and DiffExp datasets. We may then be able to generate better CalExp Masks than we can obtain from BackgroundMatchAndReject by comparing the DiffExp masks across visits in the UpdateMasks pipeline.
>
> "5. After all CalExp components have been finalized, we run the WarpRemaining and CoaddRemaining to build additional coadd data products."

These steps assume that the final visit CalExp images have already been made. The main coadd data product that is needed for DIA is the TemplateCoadd:

> "**TemplateCoadd**: A coadd data product used for difference imaging in both DRP and AP. In order to produce templates appropriate for the level of DCR in a given science image, these coadds may require a third dimension in addition to the usual two image dimen- sions (likely either wavelength or a quantity that is a function of airmass)."

Exactly how this TemplateCoadd is made is still under development: LDM-151 lists several possible scenarios, in which different algorithms for the TemplateCoadd generation are employed. The four choices for TemplateCoadd generation seem to be as follows:

* **Standard decorrelated coadds**

* **Constant-PSF partially-decorrelated coadds**

* **PSF-matched coadds**

* **"The result of inference on resampled exposures with no PSF-matching"**


## The Twinkles_subset Dataset

We'll use the `Twinkles_subset` data, in a local data "repository" copied from the shared project space:

In [ ]:
%%bash
if [ ! -d TWINKLES_DATA ]; then cp -r /project/shared/data/Twinkles_subset/input_data_v2 TWINKLES_DATA; fi

Here's what this data repository contains:

In [ ]:
!ls -lh TWINKLES_DATA/

The Butler uses a mapper to find and organize data in a format specific to each camera. Here we're using `lsst.obs.lsstSim.LsstSimMapper` mapper for the Twinkles simulated data:

In [ ]:
!cat TWINKLES_DATA/_mapper

All of the relavent images and calibrations have already been ingested into the Butler for this data set.

## Making a SkyMap

We need to make a SkyMap, to define the _tracts_ and _patches_ of sky that will be covered by our coadd image. For this, we could use the [`lsst.pipe.tasks.makeDiscreteSkyMap.py`](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/makeDiscreteSkyMap.py) command line task, as described in the [pipelines.lsst.io Getting started part 4](https://pipelines.lsst.io/getting-started/coaddition.html) tutorial.

In [ ]:
! makeDiscreteSkyMap.py TWINKLES_DATA --id --rerun coadd --config skyMap.projection="TAN"

# cf the following line from Dominique Fouchez's minicookbook:
# makeSkyMap.py input --output output --configfile makeSkyMapConfig.py

The above command fails with the following error:
```
makeDiscreteSkyMap INFO: Extracting bounding boxes of 0 images
makeDiscreteSkyMap FATAL: Failed: No data found from which to compute convex hull
```

We need to somehow specify which images in the Twinkles_subset to use - and these are e-images, rather than outputs from `processCcd`.

In [ ]:
where_is("makeDiscreteSkyMap.py", in_the="source", assuming_its_a="cmdlinetask")

Let's unpack the `makeDiscreteSkyMap` command line task, and configure and run it from python. First we need to import the task and its configuration object.

In [ ]:
from lsst.pipe.tasks.makeDiscreteSkyMap import MakeDiscreteSkyMapTask, MakeDiscreteSkyMapConfig

In [ ]:
help(MakeDiscreteSkyMapTask)

Setting up the task with the default configuration, let's run it in this vanilla mode before reconfiguring.

In [ ]:
skyMapConfig = MakeDiscreteSkyMapConfig()
skyMapTask = MakeDiscreteSkyMapTask(config=skyMapConfig)
skyMapTask.run()

## Making the Coadd Images

In [ ]:
"""
#find the tract/patches overlapping visit 182014
./reportPatchesWithImages.py output --visits 182014 --filt r
sed -e 's/^/--id filter=r /' r_182014_patches.list > patches_r.txt

#make coadd on these patches to create template
makeCoaddTempExp.py input --output output @patches_r.txt --selectId filter=r visit=182014^452599 -j 32
assembleCoadd.py output --output output @patches_r.txt --selectId filter=r visit=182014^452599 -j 32 
""";